In [ ]:

# Train and evaluate models with different configurations
for neurons_per_layer, config_label in configurations:
    for activation_function, act_label in activations:
        for optimizer, opt_label in optimizers:
            print(f"Training model with configuration: {neurons_per_layer}, activation: {act_label}, optimizer: {opt_label}")
            
            # Build and train the model
            nn_model = build_nn_model(neurons_per_layer, activation_function, optimizer)
            history = nn_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=2, 
                                   callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), 
                                              model_checkpoint_callback, csv_logger])

            # Normalize the loss values
            scaler_loss = MinMaxScaler(feature_range=(0, 1))
            normalized_train_loss = scaler_loss.fit_transform(np.array(history.history['loss']).reshape(-1, 1)).flatten()
            normalized_val_loss = scaler_loss.transform(np.array(history.history['val_loss']).reshape(-1, 1)).flatten()

            # Evaluate the model
            y_train_pred = nn_model.predict(X_train)
            y_test_pred = nn_model.predict(X_test)

            # Inverse transform to get actual values
            y_train_pred_actual = scaler_y.inverse_transform(y_train_pred).flatten()
            y_test_pred_actual = scaler_y.inverse_transform(y_test_pred).flatten()
            y_test_actual = scaler_y.inverse_transform(y_test).flatten()

            # Calculate evaluation metrics
            r2_train = r2_score(train_data['q'], y_train_pred_actual)
            r2_test = r2_score(test_data['q'], y_test_pred_actual)
            rmse_train = np.sqrt(mean_squared_error(train_data['q'], y_train_pred_actual))
            rmse_test = np.sqrt(mean_squared_error(test_data['q'], y_test_pred_actual))
            mae_train = mean_absolute_error(train_data['q'], y_train_pred_actual)
            mae_test = mean_absolute_error(test_data['q'], y_test_pred_actual)
            rme_train = np.sqrt(np.mean((y_train_pred_actual - train_data['q']) ** 2)) / np.mean(train_data['q'])
            rme_test = np.sqrt(np.mean((y_test_pred_actual - test_data['q']) ** 2)) / np.mean(test_data['q'])

            # Store results
            results.append({
                'configuration': config_label,
                'activation': act_label,
                'optimizer': opt_label,
                'r2_train': r2_train,
                'r2_test': r2_test,
                'rmse_train': rmse_train,
                'rmse_test': rmse_test,
                'mae_train': mae_train,
                'mae_test': mae_test,
                'rme_train': rme_train,
                'rme_test': rme_test
            })


            # Set Times New Roman before plotting
            plt.rcParams["font.family"] = "Times New Roman"
            sns.set(style="white", font="Times New Roman")  # or style="ticks" for a cleaner look


        # Set fixed y-axis limits (adjust these based on your data)
            y_min, y_max = -1, 1  # Example fixed range; adjust as needed

            # Plot normalized training vs testing loss curves
            plt.figure(figsize=(14, 6), dpi=300)  
            plt.plot(normalized_train_loss, color='green', linestyle='--', linewidth=5)  # Training Loss
            plt.plot(normalized_val_loss, color='purple', linewidth=5)  # Validation Loss
            plt.grid(True)

            # Customize ticks and labels
            plt.xticks(fontsize=40, fontweight='bold')
            plt.yticks(fontsize=40, fontweight='bold')
            plt.gca().tick_params(axis='both', which='major', pad=20)  # Further control over tick label spacing

            # Label Axes with Padding
            plt.xlabel('Epochs', fontsize=40, fontweight='bold', labelpad=15)  # Add more padding to x-axis label
            plt.ylabel('Loss', fontsize=40, fontweight='bold', labelpad=10) 

            # Set fixed y-axis limits
            plt.ylim(y_min, y_max)

            # Set fixed y-ticks (customize these as per your data)
            plt.yticks([y_min, (y_min + y_max) / 2, y_max], fontsize=40, fontweight='bold') 

            # Set x-axis limits from 0 to 49 (for 50 epochs)
            plt.xlim(0, 49)

            # Custom x-ticks for epochs
            plt.xticks([0, 20, 40], fontsize=40, fontweight='bold')  # First, middle, last epochs


            plt.show()  # Removed legend



           # Filter test data for actual vs predicted plot (after 2018)
            mask = test_data['date'] >= '2018-01-01'
            test_dates_filtered = test_data[mask]['date']
            y_test_actual_filtered = y_test_actual[mask]
            y_test_pred_actual_filtered = y_test_pred_actual[mask]


            # Set Times New Roman before plotting
            plt.rcParams["font.family"] = "Times New Roman"
            sns.set(style="white", font="Times New Roman")


            # Plot actual vs predicted discharge values
            plt.figure(figsize=(14, 6), dpi=300)  
            plt.plot(test_dates_filtered, y_test_actual_filtered, color='black', linewidth=5)  # Actual
            plt.plot(test_dates_filtered, y_test_pred_actual_filtered, linestyle='--', color='red', linewidth=5)  # Predicted
            plt.grid(True)

            # Customize x-ticks: Display only alternate years
            plt.xticks(fontsize=40, fontweight='bold')
            plt.gca().tick_params(axis='both', which='major', pad=10)  # Further control over tick label spacing

            # Set y-ticks to specific values
            plt.yticks([0, 3000, 6000], fontsize=40, fontweight='bold')
            plt.gca().tick_params(axis='both', which='major', pad=10)  # Further control over tick label spacing

            # Custom x-ticks: Display only alternate years
            distinct_years = test_dates_filtered.dt.year.drop_duplicates().sort_values()
            alternate_years = distinct_years[::2]  # Select every second year

            plt.xticks(
                [test_dates_filtered[test_dates_filtered.dt.year == year].iloc[0] for year in alternate_years],
                [str(year) for year in alternate_years],
                fontsize=40,
                fontweight='bold'
            )

            # Set y-axis label only
            plt.xlabel('Date', fontsize=40, fontweight='bold', labelpad=15)
            plt.ylabel('Discharge', fontsize=40, fontweight='bold', labelpad=10)

            plt.show()  # Removed legend and title